In [37]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector
import numpy as np
PATH = os.getcwd()

LOG_DIR = PATH + '/mnist-tensorboard/log1'
metadata = os.path.join(LOG_DIR, 'metadata.tsv')

mnist = tf.keras.datasets.mnist

In [38]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [39]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)
y_test = y_test.reshape(-1,1)
y_test = ohe.fit_transform(y_test)

In [40]:
images = tf.Variable(x_train, name='images')
#def save_metadata(file):
with open(metadata, 'w') as metadata_file:
    for row in range(10000):
        c = np.nonzero(y_test)[1:][0][row]
        metadata_file.write('{}\n'.format(c))



with tf.Session() as sess:
    saver = tf.train.Saver([images])

    sess.run(images.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = images.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path = metadata
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)